Перед нами стоит задача предсказания числового значения на основе заданного текста. По заголовку мы должны предсказать его числовой рейтинг от 0.0 до 10.0 с точностью 0.1. 

В качестве библиотеки, реализующей модель русского языка, мы используем spaCy (в первую очередь из соображений скорости).

- Лемматизация
- Составление триграмм
- Категоризация отрезков текста

In [4]:
import pandas as pd

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# https://spacy.io/models/ru
import spacy
nlp = spacy.load('ru_core_news_sm')

DATASETS_PATH = '/home/leo/DATASETS'

In [7]:
pd.read_csv(f"{DATASETS_PATH}/tproger.csv", index_col=0, parse_dates=['post_time', 'parse_time'])

,title,post_time,short_text,views_num,parse_time,filename
https://tproger.ru/digest/7-mobilnyh-prilozhenij-dlja-izuchenija-programmirovanija/,7 мобильных приложений для изучения программир...,2021-03-12 15:23:47+03:00,Собрали для вас актуальные мобильные приложени...,4863,2021-03-14,a8e7d0e3-07f3-589f-b037-41af38595494
https://tproger.ru/articles/moja-istorija-v-it-kak-samostojatelno-izuchit-javascript-i-python-chtoby-osushhestvit-svoju-mechtu/,Моя история в IT: как самостоятельно изучить J...,2021-03-12 14:08:22+03:00,Я закончил университет по специальности «Специ...,1197,2021-03-14,e177b763-9c88-5ab3-8879-83100f00ad33
https://tproger.ru/books/podborka-biznes-literatury-dlja-predprinimatelej/,Подборка бизнес-литературы для предпринимателей,2021-03-12 09:07:27+03:00,"Хочется верить, что прочитанные книги положите...",174,2021-03-14,325e66ce-8f7e-570c-85f6-983db7ba8786
https://tproger.ru/articles/discord-bot-na-python-dlja-avtomatizacii-raboty-s-unity-cloud-build-v-gamedev-komande/,Discord-бот на Python для автоматизации работы...,2021-03-11 18:33:48+03:00,"Я расскажу о создании Discord-бота на Python, ...",972,2021-03-14,39d92c4a-644f-5fd4-8e2a-613be518437a
https://tproger.ru/articles/11-zadach-po-sql-s-sobesedovanij-na-poziciju-data-engineer/,11 задач по SQL с собеседований на позицию Dat...,2021-03-11 11:16:34+03:00,Позиция data-инженера востребована в IT-индуст...,1518,2021-03-14,81f6f012-670a-559e-b423-e5e3998403e4
...,...,...,...,...,...,...
https://tproger.ru/articles/chto-novogo-v-java-16-i-kak-na-jeto-otreagirovali-razrabotchiki/,Что нового в Java 16 и как на это отреагировал...,2021-03-18 10:22:06+03:00,"Вчера в новости мы кратко рассказали, что в Ja...",1177,NaT,6b426022-39bb-5ba8-96a5-d3dee39b22eb
https://tproger.ru/articles/sovety-python-novichkam-ot-sinora/,Советы Python-новичкам от синьора,2021-03-26 19:43:56+03:00,Код-ревью — крайне действенный способ поддержа...,1691,NaT,6c310e45-3e77-5371-994a-dbb86e90340d
https://tproger.ru/articles/moja-istorija-v-it-ot-programmirovanija-s-otcom-do-mezhdunarodnogo-proekta/,Моя история в IT: от программирования с отцом ...,2021-03-19 08:52:13+03:00,"Мой путь в профессию начался в 14 лет, когда о...",412,NaT,fabbc69f-df3f-5ab8-ab86-3ac8436a6aca
https://tproger.ru/articles/osnovnoj-nabor-povsednevnyh-instrumentov-devops-inzhenera/,Основной набор повседневных инструментов DevOp...,2021-03-17 11:36:04+03:00,Антон Кузнецов — руководитель группы системных...,697,NaT,adcd1acb-736c-5b35-85a8-46542c5dcfca


In [8]:
pd.read_csv(f"{DATASETS_PATH}/habr.csv", index_col=0, parse_dates=['post_time', 'parse_time'])

,title,post_time,views_num,likes_num,favs_num,comments_num,parse_time,filename
https://habr.com/ru/post/529998,Самые маленькие Arduino для ваших мини-проекто...,2020-11-26 11:16:00,27000,22,102,23,2021-03-24 00:00:00.000000,b22cc944-ccf4-5000-a939-bb6e79811832
https://habr.com/ru/post/529996,На GitHub введут темный режим спустя 7 лет пос...,2020-11-26 10:46:00,3800,14,4,24,2021-03-24 00:00:00.000000,5385f4e5-f5b6-5f6d-aad2-43eb26fb1989
https://habr.com/ru/post/529992,Внутренний стартап: как мы делали продукт в се...,2020-11-26 10:59:00,1400,7,16,2,2021-03-24 00:00:00.000000,3c18941f-1517-5f84-b7ff-e0e9b2105dc6
https://habr.com/ru/post/529990,Подставка DIY для ноутбука и ленивых,2020-11-26 21:48:00,6700,10,23,20,2021-03-24 00:00:00.000000,88750ed7-c0ed-51b0-932f-7e7df773ac9e
https://habr.com/ru/post/529988,Сравнение аналитических систем для мобильного ...,2020-11-26 10:30:00,1200,9,12,1,2021-03-24 00:00:00.000000,f8e68362-373c-541e-a5ab-383108f509bc
...,...,...,...,...,...,...,...,...
https://habr.com/ru/company/itsumma/blog/549726/,"Свою квалификацию программиста можно повысить,...",2021-03-30 10:54:00,1400,5,17,2,2021-03-30 17:01:24.028835,f9b64124-791b-5869-89f5-d3e2564bf65b
https://habr.com/ru/company/kingston_technology/blog/549492/,Kingston NV1 NVMe SSD: твердотельный накопител...,2021-03-29 11:30:00,2700,3,7,5,2021-03-30 17:01:32.463173,b5eeb9f3-bcb4-5c39-bde6-b6b7fbff1207
https://habr.com/ru/company/mailru/blog/548480/,Полное руководство по созданию Docker-образа д...,2021-03-24 08:23:00,5000,34,132,4,2021-03-30 17:01:54.823272,05e1f5c4-a38e-597b-9fba-fa71c3402d42
https://habr.com/ru/company/sberbank/blog/549662/,Process Mining как эволюция «научного управлен...,2021-03-30 11:03:00,333,6,7,0,2021-03-30 17:02:02.625687,dfbb042b-7396-5d08-9ab9-980d2cadc732
